# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [ ]:
# your code here
orders=pd.read_csv('Orders.csv')

In [4]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [36]:
# your code here
spent= orders.groupby('CustomerID').agg('sum')['amount_spent'].sort_values(ascending=False).reset_index()
spent.head()

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06


In [42]:
q95 = spent['amount_spent'].quantile(0.95)
q75 = spent['amount_spent'].quantile(0.75)
print(q95,q75)

5840.181999999983 1661.6400000000003


In [138]:
Preferred = spent[(spent['amount_spent']>=q75)&(spent['amount_spent']<q95)]
VIP = spent[(spent['amount_spent']>=q95)]
display(Preferred.head())
display(VIP.head())

,CustomerID,amount_spent,Customer Type
217,13050,5836.86,Preferred
218,12720,5781.73,Preferred
219,15218,5756.89,Preferred
220,17686,5739.46,Preferred
221,13178,5725.47,Preferred


,CustomerID,amount_spent,Customer Type
0,14646,280206.02,VIP
1,18102,259657.30,VIP
2,17450,194550.79,VIP
3,16446,168472.50,VIP
4,14911,143825.06,VIP


In [139]:
def typecustomer(i):
    if i>=q75 and i<q95:
        return 'Preferred'
    elif i>=q95:
        return 'VIP'
    else:
        return 'Regular'
        
spent['Customer Type']=spent['amount_spent'].apply(typecustomer)
spent.head(1)

,CustomerID,amount_spent,Customer Type
0,14646,280206.02,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [130]:
# your code here
y=orders[['CustomerID','Country']].drop_duplicates()
x = VIP.merge(y, how='left', on='CustomerID')[['CustomerID','Country']]
x.drop_duplicates(inplace=True)
print(x.shape)
display(x['Country'].value_counts().head(1))

(218, 2)


United Kingdom    177
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [132]:
# your code here
VIP_preferred = VIP.merge(Preferred, how='outer')
z = VIP_preferred.merge(y, how='left', on='CustomerID')[['CustomerID','Country']]
z.drop_duplicates(inplace=True)
print(z.shape)
display(z['Country'].value_counts().head())

(1091, 2)


United Kingdom    932
Germany            39
France             29
Belgium            12
Switzerland         9
Name: Country, dtype: int64